In [53]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [54]:
df_final = pd.read_csv('./final_dfs/df_final_without_labor_with_clusters.csv', converters={'ags2': str, 'ags5': str})
company_changes_date = pd.read_csv('./final_dfs/company_changes_by_date.csv', converters={'ags2': str, 'ags5': str, 'date':str})
labor_market_by_date = pd.read_csv('./final_dfs/labor_market_by_date.csv', converters={'ags2': str, 'ags5': str, 'date':str})
labor_market_region = pd.read_csv('./final_dfs/labor_market_region.csv', converters={'ags2': str, 'ags5': str, 'date':str})

In [55]:
labor_market_by_date.date

0        2020-01-01
1        2020-02-01
2        2020-03-01
3        2020-04-01
4        2020-05-01
            ...    
89418    2020-12-01
89419    2021-01-01
89420    2021-02-01
89421    2021-03-01
89422    2021-04-01
Name: date, Length: 89423, dtype: object

In [56]:
company_changes_date

,ags2,ags5,date,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,number_of_companies_domestic_staff,number_of_companies_economic_services,...,number_of_companies_real_estat,number_of_companies_rendering_other_services,number_of_companies_repair_motor_vehicles,number_of_companies_technical_services,number_of_companies_transport,number_of_companies_unknown_sector,number_of_companies_water_and_sewage,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups
0,01,01001,2019-01-01,34.0,14.0,108.0,131.0,264.0,0.0,130.0,...,279.0,1089.0,866.0,619.0,105.0,183.0,12.0,15.0,6.0,5.0
1,01,01001,2019-02-01,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,11.0,2.0,15.0
2,01,01001,2019-03-01,34.0,12.0,87.0,132.0,207.0,0.0,246.0,...,293.0,568.0,797.0,524.0,103.0,68.0,6.0,6.0,3.0,14.0
3,01,01001,2019-04-01,43.0,9.0,119.0,98.0,170.0,0.0,215.0,...,207.0,876.0,678.0,404.0,72.0,60.0,9.0,9.0,2.0,14.0
4,01,01001,2019-05-01,44.0,12.0,146.0,150.0,242.0,0.0,281.0,...,315.0,966.0,923.0,614.0,116.0,46.0,10.0,8.0,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11251,16,16077,2020-12-01,115.0,99.0,158.0,44.0,457.0,1.0,246.0,...,100.0,1212.0,728.0,385.0,134.0,110.0,26.0,2.0,1.0,6.0
11252,16,16077,2021-01-01,115.0,99.0,158.0,44.0,459.0,1.0,246.0,...,99.0,1212.0,729.0,384.0,134.0,110.0,26.0,0.0,3.0,2.0
11253,16,16077,2021-02-01,115.0,99.0,158.0,43.0,457.0,1.0,245.0,...,97.0,1210.0,722.0,385.0,134.0,110.0,26.0,0.0,0.0,5.0
11254,16,16077,2021-03-01,114.0,100.0,158.0,42.0,455.0,1.0,245.0,...,96.0,1208.0,723.0,388.0,133.0,108.0,26.0,4.0,0.0,4.0


In [57]:
df_final = df_final.merge(company_changes_date, left_on=['ags2','ags5'], right_on=['ags2','ags5'])
df_final = df_final.merge(labor_market_by_date, left_on=['ags2','ags5','date'], right_on=['ags2','ags5','date'])
df_final = df_final.merge(labor_market_region, left_on=['ags5'], right_on=['ags5'])

In [58]:
df_final = df_final.drop(['Unnamed: 0_x', 'data_index'], 1)

In [59]:
df_final

,cluster,kreis,ags5,ags2,supermarkets_population,supermarkets_average_distance,public_transport_availability,average_distance_bus_stop,average_distance_train_station,average_distance_public_transport,...,number_of_companies_water_and_sewage_y,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,Unnamed: 0_y,bundesland,variable,value,labor_market_region_number,labor_market_region_name
0,2,Regionalverband Saarbrücken,10041,10,76,734,52,260,1804,255,...,79.0,27.0,33.0,37.0,70966,Saarland,unemployed,14584.00000,171,Saarbrücken
1,2,Regionalverband Saarbrücken,10041,10,76,734,52,260,1804,255,...,79.0,27.0,33.0,37.0,70994,Saarland,unemployment_rate,8.40000,171,Saarbrücken
2,2,Regionalverband Saarbrücken,10041,10,76,734,52,260,1804,255,...,79.0,27.0,33.0,37.0,71034,Saarland,underemployment_without_short_time _work,22351.00000,171,Saarbrücken
3,2,Regionalverband Saarbrücken,10041,10,76,734,52,260,1804,255,...,79.0,27.0,33.0,37.0,71058,Saarland,unemployment_benefit_recipients,3161.00000,171,Saarbrücken
4,2,Regionalverband Saarbrücken,10041,10,76,734,52,260,1804,255,...,79.0,27.0,33.0,37.0,71082,Saarland,registerd_jobs,3056.00000,171,Saarbrücken
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18504,0,Altenburger Land,16077,16,62,1458,35,584,3628,574,...,26.0,1.0,1.0,4.0,89231,Thüringen,displayed_short_time_work_people,54.00000,223,Altenburger Land
18505,0,Altenburger Land,16077,16,62,1458,35,584,3628,574,...,26.0,1.0,1.0,4.0,89279,Thüringen,unemployed,3078.00000,223,Altenburger Land
18506,0,Altenburger Land,16077,16,62,1458,35,584,3628,574,...,26.0,1.0,1.0,4.0,89307,Thüringen,unemployment_rate,6.90000,223,Altenburger Land
18507,0,Altenburger Land,16077,16,62,1458,35,584,3628,574,...,26.0,1.0,1.0,4.0,89395,Thüringen,registerd_jobs,769.00000,223,Altenburger Land


In [60]:
#df_final.to_csv('./final_dfs/df_final_with_dates.csv', index = False)